In [2]:
import numpy as np
import gvar as gv
import re 
import pandas as pd 
import sys
import copy
import tables as h5
import h5py
import os 
import time
import re
sys.path.insert(0, '/home/gbradley/nucleon_elastic_FF')
from nucleon_elastic_ff.data.h5io import get_dsets 


In [6]:
h5fname = '/home/gbradley/c51_corr_analysis/tests/data/C13/C13-b_4002.ama.h5'


   
with h5py.File(h5fname, 'r') as h5f:
    # top_keys = [key for key in h5f.keys()]
    # for top_key in top_keys:
    #     group = h5f[top_key]
    #     group['10'] = group['3pt_tsep10']
    data = {}
    dsets = get_dsets(h5f)
    # print(dsets)
    for key in dsets.keys():
        # print(key)
        data[key] = h5f[key][:]
    print(data['2pt/ext_current/src5.0_snk5.0/ext_axial_A1_A1/C13.b_4002/AMA'])
    # [val for key,val in dsets.items() if key in mystring]
    # print(val[key])
    # for key in dsets.keys():
    #     if key in 
        
            
    #     dset = ifile['data']
    #     for key in dset.keys():
    #         data[key] = ifile['data'][key][:]
    # for key in ['13','14','15','16']:
    #     data[int(key)] = data.pop(key)

    # return data

[2022-08-07 16:57:06,006|lqcd correlator analysis@INFO] Locating all dsets of h5 file `/home/gbradley/c51_corr_analysis/tests/data/C13/C13-b_4002.ama.h5`


[(-2.58211152e-04, -1.30685940e-15) ( 5.87860783e-05,  1.41900927e-14)
 ( 2.57692208e-05, -2.63309518e-15) ( 1.13296507e-05,  4.67235660e-16)
 ( 5.59413382e-06,  3.97957074e-15) ( 2.38318088e-06,  3.10596431e-15)
 ( 1.14248262e-06,  2.60821298e-16) ( 5.90481965e-07, -9.83484766e-16)
 ( 1.60438565e-07, -6.76532309e-16) ( 7.70157259e-08, -1.51733247e-15)
 ( 1.56569742e-07,  3.28791731e-17) ( 6.94757076e-08, -7.04444216e-16)
 ( 1.97736845e-08,  2.10743094e-16) (-2.90964137e-08, -7.25096198e-17)
 (-2.23251868e-08,  4.06067857e-17) (-9.99152606e-09,  1.87443733e-16)
 (-6.44054813e-10,  2.11968486e-16) ( 2.47886818e-09,  1.49763737e-16)
 ( 2.23477394e-09,  9.59116581e-17) (-4.31796012e-10, -4.06884055e-17)
 (-1.01335198e-08,  1.59745027e-16) (-5.28003747e-09,  3.49200185e-17)
 ( 3.48499936e-09, -8.23085473e-17) (-1.13816400e-08,  5.71131949e-17)]


In [22]:

# import fitter.corr_functions as cf
# import fitter.fit_twopt

directory = '/home/gbradley/c51_corr_analysis/tests/data/C13/'
N_cnf = len([name for name in os.listdir(directory) if os.path.isfile(name)])

dirs = os.listdir( directory )

cnf_abbr = [files.split(".ama.h5",0) for files in dirs]

# data_file_list = os.path.realpath(dirs)
data_file_list = list()
for dirpath,_,filenames in os.walk(directory):
    for f in filenames:
        data_file_list.append(os.path.abspath(os.path.join(dirpath, f)))
file = data_file_list[0]


NUCL: nucleon
U: quark bilinear operator inserted on up-quark; D will be used for down-quark
MIXED: "mixed" type of spin projection is used
NONREL: non-relativistic proton is used
l0:  when inserting the quark bilinear oprator, the separation of the quarks of the bilinear operator is zero (local operator); you might see some l1 (quark bilinear operator separated by 1 lattice unit) data as well
g13: the gamma matrix of the quark bilinear operator is "13" in Chroma convention. Page 6 and 7 of the attached pdf shows the Chroma gamma matrix convention and its indexing; their indexing is summarized below:
 
0: scalar; I
15: pseudoscalar; g_5
1: vector;  g_x
2: vector;  g_y
4: vector;  g_z
8: vector;  g_t
14: axial;   g_x g_5
13: axial;  -g_y g_5
11: axial;   g_z g_5
7: axial;  -g_t g_5
9: tensor;  g_x g_t
10: tensor;  g_y g_t
12: tensor;  g_z g_t
3: tensor;  g_x g_y
6: tensor;  g_y g_z
5: tensor;  g_x g_z

In [23]:


string = (
    "3pt_tsep12/NUCL_D_MIXED_NONREL_l0_g0/src5.0_snk5.0/qz+0_qy+0_qx+0/C13.b_5682/AMA"
)

patterns = [
    "3pt",
    "_tsep(?P<tsep>[0-9]|[0-9]+)",  # must match `_tsep` and stores the following numbers (any length)
    "/NUCL_(?P<quark>U|D)",  # Store U or D in quark
    "_MIXED_NONREL",  # Not sure if this changes. Not stored for now
    "_l(?P<l>[0-9]+)",  # action parameters?
    "_g(?P<g>[0-15]+)",
    "/src(?P<src>[0-9\.]+)",  # Stores numbers + . to store decimals. Must escape .
    "_snk(?P<snk>[0-9\.]+)",  # Stores numbers + . to store decimals. Must escape .
    "/qz(?P<qz>[\+\-0-9]+)", 
    "_qy(?P<qy>[\+\-0-9]+)", 
    "_qx(?P<qx>[\+\-0-9]+)", 
    
]

for n in range(len(patterns)):
    pattern = "".join(patterns[:n+1])
    match = re.match(pattern, string)
    if not match:
        print(pattern)
        break

if match:
    print(match.groupdict())
# 3pt_tsep8/NUCL_U_MIXED_NONREL_l0_g9/src5.0_snk5.0/qz-3_qy+0_qx+1/C13.b_5682/AMA
# 2pt/ext_current/src5.0_snk5.0/ext_axial_A1_A1/C13.b_5682/AMA
# 2pt/ext_current/src5.0_snk5.0/ext_axial_A1_A1_px1_py0_pz0/C13.b_5682/AMA
# 2pt/ext_current/src5.0_snk5.0/ext_axial_A3_P_px1_py0_pz0/C13.b_5682/AMA
# 2pt/ext_current/src5.0_snk5.0/ext_axial_A4_A4_px3_py1_pz0/C13.b_5682/AMA
# 2pt/ext_current/src5.0_snk5.0/ext_axial_A4_P/C13.b_5682/AMA
# 2pt/ext_current/src5.0_snk5.0/ext_axial_A4_P_px1_py0_pz0/C13.b_5682/AMA
# 2pt/ext_current/src5.0_snk5.0/ext_vector_T12_T12_px1_py0_pz0/C13.b_5682/AMA
# 2pt/ext_current_SP/src5.0_snk5.0/ext_vector_V2_V2_px1_py1_pz1/C13.b_5682/AMA
# 2pt/pion/src5.0_snk5.0/pion_px1_py0_pz0/C13.b_5682/AMA
# 2pt/pion_SP/src5.0_snk5.0/pion_px1_py0_pz0/C13.b_5682/AMA
# 2pt/proton/src5.0_snk5.0/proton_px1_py0_pz0/C13.b_5682/AMA
# 2pt/proton_SP/src5.0_snk5.0/proton_px1_py0_pz0/C13.b_5682/AMA

{'tsep': '12', 'quark': 'D', 'l': '0', 'g': '0', 'src': '5.0', 'snk': '5.0', 'qz': '+0', 'qy': '+0', 'qx': '+0'}


In [96]:
def parse_baryon_tag(datatag):
    datatag_split = datatag.split('/')
    corr_type     = datatag_split[0]
    tsep          = int(corr_type.split('_tsep')[1])
    buffer        =  datatag_split[1]
    channel       = buffer.split('_')[0]
    quark_ins       = buffer.split('_')[1]
    spin_proj       = buffer.split('_')[2]
    quark_sep       = buffer.split('_')[3]
    gamma           = buffer.split('_')[4] #gamma matrix of quark bilinear operator in the CHROMA convention , value accessed via dict
    src_snk_sep     = datatag_split[2]
    mom         = datatag_split[3]
    mom0       = mom.split('_')[0]
    mom1       = mom.split('_')[1]
    mom2       = mom.split('_')[2]
    momentum        = (mom0,mom1,mom2)
    config   = datatag_split[4]

    data_dict = dict()
    data_dict['corr_type']   = corr_type
    data_dict['tsep']        = tsep
    data_dict['buffer']      = buffer
    data_dict['channel']     = channel
    data_dict['quark_ins']   = quark_ins
    data_dict['spin_proj']   = spin_proj
    data_dict['quark_sep']   = quark_sep
    data_dict['gamma']       = gamma
    data_dict['src_snk_sep'] = src_snk_sep
    data_dict['mom']         = momentum
    data_dict['config']      = config
    return data_dict

   


In [97]:
parse_baryon_tag(string)

{'corr_type': '3pt_tsep12',
 'tsep': 12,
 'buffer': 'NUCL_D_MIXED_NONREL_l0_g0',
 'channel': 'NUCL',
 'quark_ins': 'D',
 'spin_proj': 'MIXED',
 'quark_sep': 'NONREL',
 'gamma': 'l0',
 'src_snk_sep': 'src5.0_snk5.0',
 'mom': ('qz+0', 'qy+0', 'qx+0'),
 'config': 'C13.b_5682'}

In [24]:
columns = ["tsep", "quark", "l", "g", "src", "snk","qz","qy","qx"]


In [68]:
data_frames = []

with h5py.File(file, "r") as h5f:
    dsets = get_dsets(h5f)
    # print(dsets)
    for key, dset in dsets.items():
        match = re.search(pattern, string)
        if match:
            info = match.groupdict()
            # print(info)
#             corr = info.pop("tsep")

            quark = info.pop("quark")
            # print(quark)
            info["quark"] = quark[0]
# #             # info["parity"] = -1 if len(nucleon_parity) == 2 else 1
            
            gamma = info.pop("g")
            if gamma in ["g1","g2","g4","g8"]:
                info["gamma"] = "vector"
            elif gamma in ["g0"]:
                info["gamma"] = "scalar"
            elif gamma in ["g5"]:
                info["gamma"] = "pseudoscalar"
            elif gamma in ["g14","g13","g11","g7"]:
                info["gamma"] = "axial"
            elif gamma in ["g14","g13","g11","g7"]:
                info["gamma"] = "axial"
            elif gamma in ["g9","g10","g12","g3","g6","g5"]:
                info["gamma"] = "tensor"

            # current_key = key.replace("g", "")
            curr_dset = h5f[key]

            cfgs = dset[:]
            corr = (
                curr_dset[()].real 
                # if info["current"] in ["V4"] else curr_dset[()].imag
            )
            # print(corr.shape[-1])
            ts = range(corr.shape[-1])
            # print(ts)
            tmp_df = (
                pd.DataFrame(index=cfgs, columns=ts, data=corr)
                .unstack()
                .reset_index()
                .rename(columns={"level_0": "tsep", "level_1": "cfg", 0: "corr"})
            )
            # data_frames = {}
            ydict = {}
            for key, val in info.items():
                tmp_df[key] = val
            data_frames.append(tmp_df.astype({"tsep": int}))
            print(data_frames)


# df = pd.concat(
#     data_frames, 
#     ignore_index=True, 
# ).reindex(columns, axis=1).sort_values(columns).reset_index(drop=True)
# print(df.type())
# # df.head()
# print(df.keys())
# for 'tsep' in df.keys():
#     if not isinstance(tsep, int):
#                 raise TypeError("t_sink keys must be integers.")

[2022-08-04 13:40:32,887|nucleon_elastic_ff@INFO] Locating all dsets of h5 file `/home/gbradley/c51_corr_analysis/tests/data/C13/C13-b_5682.ama.h5`


[(-2.56356869e-04, -1.11207678e-14) ( 6.19844931e-05, -5.57827152e-15)
 ( 2.80756822e-05, -4.45528297e-15) ( 1.28634833e-05,  1.58291252e-15)
 ( 5.51984960e-06, -1.83916225e-17) ( 2.06134187e-06, -1.83563147e-15)
 ( 8.22243347e-07,  1.16850159e-15) ( 3.55047819e-07,  2.13496274e-15)
 (-8.82941188e-08,  1.21976550e-15) (-1.26974504e-07, -7.16024162e-16)
 (-1.10442904e-07,  5.56118423e-16) (-8.31974471e-08,  1.46340611e-16)
 ( 3.73264712e-08, -2.94597794e-16) ( 9.58739622e-08,  1.66116448e-16)
 ( 8.90409021e-08,  3.10279445e-16) ( 1.33651669e-08, -2.56066583e-16)
 ( 6.61094425e-08, -1.63885921e-16) (-4.16672070e-08,  3.17653449e-16)
 (-7.37063703e-08, -3.28802309e-16) (-5.32416570e-08,  6.22143052e-17)
 (-3.24150229e-08,  1.05534429e-16) (-3.21676882e-09, -3.78036628e-17)
 (-4.83421180e-09, -8.44932836e-17) (-1.37609513e-08, -6.75218136e-17)]


IndexError: list index out of range

In [46]:
import corr_functions as cf 
print(df.tsep)
ydict = {tag: val for tag, val in df.items() if isinstance(tag, int)}
ydict


0          12
1          12
2          12
3          12
4          12
           ..
3729211    12
3729212    12
3729213    12
3729214    12
3729215    12
Name: tsep, Length: 3729216, dtype: int64


{}

## statistical average ##


In [ ]:
def avg_data(arg):
    corr_avg = gvar.dataset.avg_data(
        arg.pivot(index="cfg", columns="t", values="corr").values
    )
    return pd.Series(corr_avg)


group = isospin_spin_parity_avg_df.groupby(["nucleon", "current", "tsep"])
corr_df = (
    group.apply(avg_data)
    .reset_index(level=-1)
    .rename(columns={"level_3": "t", 0: "corr"})
    .reset_index()
    .set_index(["nucleon", "current", "tsep", "t"])
)

corr_df.head()

## momentum average ##

In [ ]:
def mom_avg(h5_data,state,mom_lst,weights=False):
    '''
    perform a momentum average of a state from an open h5 file
    data file is assumed to be of shape [Nt,Nz,Ny,Nx,[re,im]]
    data_mom = h5_data[state][:,pz,py,px]
    '''
    d_lst = []
    w = []
    for mom in mom_lst:
        px,py,pz = mom['momentum']
        w.append(mom['weight'])
        #print(state)
        d_lst.append(h5_data[state][:,pz,py,px])
    d_lst = np.array(d_lst)
    w = np.array(w)
    if weights:
        for wi,we in enumerate(w):
            d_lst[wi] = we*d_lst[wi]
        d_avg = np.sum(d_lst,axis=0) / np.sum(w)
    else:
        d_avg = np.mean(d_lst,axis=0)
    return d_avg
# mom_avg('/home/gbradley/c51_corr_analysis/tests/data/C13/C13-b_5178.ama.h5', state, mom_lst)

mom_lst = []
for px in range(-2,3):
    for py in range(-2,3):
        for pz in range(-2,3):
            if px**2 + py**2 + pz**2 <= 5:
                mom_lst.append('pz'+str(pz)+'_py'+str(py)+'_px'+str(px))